In [5]:
#importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import json
from Utils.evaluation import EvaluationMetrics, ActualStats
from Utils.helper import *
from Utils.sample_squads import *

## Innings 1

In [6]:
model_inn1 = tf.keras.models.load_model('Models\Inn1-HeavyDense-ep20to50\cp-0029.h5')
model_inn2 = tf.keras.models.load_model('Models\Inn2-HeavyDenseRequired-ep30to50\cp-0016.h5')

In [7]:
evaluator = EvaluationMetrics(model_inn1, model_inn2,
                              # )
                              "Evaluation/tournament1.pkl")

In [9]:
for _ in range(1):
    print(f"\n\nMatch number: {evaluator.match_count+1}")
    evaluator.simulate_match(1)
evaluator.display_table()
evaluator.save_object("Evaluation/tournament1.pkl")



Match number: 57
All Matches in the series are over


,Played,Wins,Losses,Points,NRR
Delhi Capitals,14,10,4,20,0.381699
Kings XI Punjab,14,9,4,19,0.427534
Kolkata Knight Riders,14,9,5,18,0.376469
Chennai Super Kings,14,7,7,14,-0.017489
Royal Challengers Bangalore,14,6,8,12,-0.187438
Mumbai Indians,14,5,8,11,-0.099685
Rajasthan Royals,14,5,9,10,-0.221348
Sunrisers Hyderabad,14,4,10,8,-0.665983


In [12]:
evaluator = ActualStats("Evaluation/actual.pkl")
# actualstat = ActualStats()
# actualstat.run_df(1, True)
# actualstat.run_df(2, True)

In [13]:
batsmen_result_dic = {}
bat_stat = evaluator.batsmen_stat
for batsman in bat_stat:
    tot_runs = 0
    tot_balls = 0
    num_4 = 0
    num_6 = 0
    num_inn = 0
    num_out = 0
    hs = 0
    dismissal_type = []
    dismissed_by = []
    for i in bat_stat[batsman]:
        num_inn += 1
        tot_runs += i["Runs"]
        if i["Runs"] > hs:
            hs = i["Runs"]
        tot_balls += i["Balls Faced"]
        num_4 += i["Fours"]
        num_6 += i["Sixes"]
        if i["Dismissal Type"] != "Not Out":
            num_out += 1
        dismissal_type.append(i["Dismissal Type"])
        dismissed_by.append(i["Dismissed By"])
    batsmen_result_dic[batsman] = {
        "Innings": num_inn,
        "Runs": tot_runs,
        "Balls": tot_balls,
        "Fours": num_4,
        "Sixes": num_6,
        "High Score": hs,
        "Average": tot_runs/num_out if num_out!=0 else tot_runs,
        "Strike Rate": tot_runs/tot_balls*100 if tot_balls!=0 else 0,
        "Boundry Percent": (6*num_6+4*num_4)/tot_runs*100 if tot_runs!=0 else 0,
        }

In [14]:
batsmen_stat_df = pd.DataFrame.from_dict(batsmen_result_dic, orient='index')
save_df = batsmen_stat_df.sort_values(by=["Runs", "Balls"], ascending=False)
# save_df.to_csv("Evaluation/Evaluate_Batsmen.csv")
save_df.head(20)

,Innings,Runs,Balls,Fours,Sixes,High Score,Average,Strike Rate,Boundry Percent
V Kohli,181,5709,4401,499,204,109,31.541436,129.720518,56.402172
SK Raina,186,5284,3845,501,210,100,28.408602,137.425228,61.771385
RG Sharma,194,5164,3944,464,221,109,26.618557,130.933063,61.618900
DA Warner,137,5094,3586,510,203,126,37.182482,142.052426,63.957597
S Dhawan,168,5057,3956,590,116,106,30.101190,127.831143,60.431086
AB de Villiers,153,4815,3168,393,239,133,31.470588,151.988636,62.429907
MS Dhoni,178,4549,3316,320,227,83,25.556180,137.183353,58.078699
CH Gayle,126,4536,3077,379,337,175,36.000000,147.416315,77.998236
RV Uthappa,178,4532,3490,458,174,87,25.460674,129.856734,63.459841
G Gambhir,145,4102,3307,486,66,93,28.289655,124.039915,57.045344


In [15]:
bowler_result_dic = {}
bowler_stat = evaluator.bowler_stat
for bowler in bowler_stat:
    tot_runs = 0
    tot_balls = 0
    tot_wickets = 0
    for i in bowler_stat[bowler]:
        tot_runs += i["Runs Conceded"]
        tot_balls += i["Balls"]
        tot_wickets += i["Wickets Taken"]
    bowler_result_dic[bowler] = {
        "Runs Conceded": tot_runs,
        "Overs": f"{tot_balls//6}.{tot_balls%6}",
        "Wickets": tot_wickets,
        "Economy": tot_runs/tot_balls*6 if tot_balls != 0 else 0,
        }


In [16]:
bowler_stat_df = pd.DataFrame.from_dict(bowler_result_dic, orient='index')
save_df = bowler_stat_df.sort_values(by=["Wickets", "Economy"], ascending=[False, True])
# save_df.to_csv("Evaluation/Evaluate_Bowler.csv")
save_df.head(20)

,Runs Conceded,Overs,Wickets,Economy
SL Malinga,3366,471.1,170,7.143969
A Mishra,3800,517.3,152,7.342995
Harbhajan Singh,3967,562.2,150,7.054535
DJ Bravo,3796,450.5,147,8.419963
PP Chawla,4115,524.0,146,7.853053
B Kumar,3043,422.3,129,7.202367
SP Narine,3055,448.5,123,6.806535
R Ashwin,3533,512.1,123,6.898145
UT Yadav,3525,412.4,116,8.542003
RA Jadeja,3459,451.0,110,7.669623
